In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
import xgboost as xgb
import joblib


# EDA

In [4]:
pd.set_option('display.max_columns', 50)
data = pd.read_csv("Kepler_data.csv")
data.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,35.8,1.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,25.8,2.0,q1_q17_dr25_tce,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,76.3,1.0,q1_q17_dr25_tce,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,505.6,1.0,q1_q17_dr25_tce,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,40.9,1.0,q1_q17_dr25_tce,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [5]:
data.columns
data.shape

(9564, 27)

In [6]:
data.isna().sum()

kepid                   0
kepoi_name              0
kepler_name          6817
koi_disposition         0
koi_pdisposition        0
koi_score            1510
koi_fpflag_nt           0
koi_fpflag_ss           0
koi_fpflag_co           0
koi_fpflag_ec           0
koi_period              0
koi_time0bk             0
koi_impact            363
koi_duration            0
koi_depth             363
koi_prad              363
koi_teq               363
koi_insol             321
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_steff             363
koi_slogg             363
koi_srad              363
ra                      0
dec                     0
koi_kepmag              1
dtype: int64

In [7]:
unrelated_features = ['koi_disposition', 'koi_pdisposition','koi_tce_delivname','kepler_name']
data_cleaned = data.drop(unrelated_features, axis=1)

In [8]:
data_cleaned = data_cleaned.dropna()

In [9]:
data_cleaned.isna().sum()

kepid               0
kepoi_name          0
koi_score           0
koi_fpflag_nt       0
koi_fpflag_ss       0
koi_fpflag_co       0
koi_fpflag_ec       0
koi_period          0
koi_time0bk         0
koi_impact          0
koi_duration        0
koi_depth           0
koi_prad            0
koi_teq             0
koi_insol           0
koi_model_snr       0
koi_tce_plnt_num    0
koi_steff           0
koi_slogg           0
koi_srad            0
ra                  0
dec                 0
koi_kepmag          0
dtype: int64

In [10]:
data_cleaned.head()

,kepid,kepoi_name,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,10797460,K00752.01,1.000,0,0,0,0,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,35.8,1.0,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,10797460,K00752.02,0.969,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,25.8,2.0,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,10811496,K00753.01,0.000,0,0,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,76.3,1.0,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,10848459,K00754.01,0.000,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,505.6,1.0,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,10854555,K00755.01,1.000,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,40.9,1.0,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [11]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7994 entries, 0 to 9563
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   kepid             7994 non-null   int64  
 1   kepoi_name        7994 non-null   object 
 2   koi_score         7994 non-null   float64
 3   koi_fpflag_nt     7994 non-null   int64  
 4   koi_fpflag_ss     7994 non-null   int64  
 5   koi_fpflag_co     7994 non-null   int64  
 6   koi_fpflag_ec     7994 non-null   int64  
 7   koi_period        7994 non-null   float64
 8   koi_time0bk       7994 non-null   float64
 9   koi_impact        7994 non-null   float64
 10  koi_duration      7994 non-null   float64
 11  koi_depth         7994 non-null   float64
 12  koi_prad          7994 non-null   float64
 13  koi_teq           7994 non-null   float64
 14  koi_insol         7994 non-null   float64
 15  koi_model_snr     7994 non-null   float64
 16  koi_tce_plnt_num  7994 non-null   float64
 17  

# Model Training

In [ ]:
# Training

# Features & Target
X = data_cleaned.drop(columns=["koi_score",'kepid','kepoi_name'])  # drop object column
y = data_cleaned["koi_score"]


# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# XGBoost Regressor (for GridSearchCV)
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)


# Parameter Grid
param_grid = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0]
}


# GridSearchCV Setup
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,                               # 5-fold cross-validation
    scoring='neg_root_mean_squared_error',  
    verbose=1,
    n_jobs=-1
)

# Train
grid_search.fit(X_train, y_train)

# Best Model
best_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)
best_model.save_model("xgb_koi_model.json")  # saves in JSON format


# Predictions
y_train_pred = best_model.predict(X_train)
y_test_pred  = best_model.predict(X_test)


# Errors / Metrics
train_rmse = root_mean_squared_error(y_train, y_train_pred)
test_rmse  = root_mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2  = r2_score(y_test, y_test_pred)

# print("Train RMSE:", train_rmse, "| Train R²:", train_r2)
# print("Test RMSE :", test_rmse,  "| Test R² :", test_r2)

joblib.dump(best_model, "xgb_koi_model.pkl")



Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500, 'subsample': 0.6}


['xgb_koi_model.pkl']